In [ ]:
# !pip install tensorflow
# !pip install keras
# !pip install sklearn
# !pip install theano

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import random
from tensorflow import keras
import numpy as np
# from tensorflow.keras import layers, Dense, Input, InputLayer, Flatten
# from tensorflow.keras.models import Sequential, Model


from keras.models import Sequential
from keras import layers
from keras.layers import Dense,Input, InputLayer, Flatten


from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline



from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
# from keras.optimizers import  adam
from tensorflow.keras.optimizers import SGD,RMSprop, Adam

from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random 
from numpy import *
from PIL import Image
import theano

Using TensorFlow backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [7]:
path_test = "./Datasets/training-data/"

In [8]:
CATEGORIES = ['bright_dune', 'crater','dark_dune','impact_ejecta','slope_streak']

In [9]:
training = []

def createTrainingData():
  for category in CATEGORIES:
    path = os.path.join(path_test, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img))
      new_array = cv2.resize(img_array, (227, 227))
      training.append([new_array, class_num])
      
createTrainingData()

In [10]:
random.shuffle(training)
X = []
y = []

for features, label in training:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, 227, 227, 3)
y = np.array(y)

In [11]:
X.shape

(11919, 227, 227, 3)

In [12]:
X = X.astype('float32')
X /= 255

y

array([4, 0, 3, ..., 4, 2, 0])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [14]:
batch_size = 20
nb_classes = 5
nb_epochs = 10
img_rows, img_columns = 227, 227
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='same', activation=tf.nn.relu,
                           input_shape=(227, 227, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((3, 3), strides=2),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same', activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((3, 3), strides=2),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5,  activation=tf.nn.softmax)
])


In [16]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))

Train on 9535 samples, validate on 2384 samples
Epoch 1/10
9535/9535 [==============================] - 52s 5ms/sample - loss: 3.7109 - accuracy: 0.5721 - val_loss: 1.5428 - val_accuracy: 0.4698
Epoch 2/10
9535/9535 [==============================] - 41s 4ms/sample - loss: 1.2737 - accuracy: 0.6327 - val_loss: 1.6343 - val_accuracy: 0.4572
Epoch 3/10
9535/9535 [==============================] - 40s 4ms/sample - loss: 0.9806 - accuracy: 0.7053 - val_loss: 2.4515 - val_accuracy: 0.5285
Epoch 4/10
9535/9535 [==============================] - 40s 4ms/sample - loss: 0.7240 - accuracy: 0.7662 - val_loss: 0.8662 - val_accuracy: 0.7282
Epoch 5/10
9535/9535 [==============================] - 41s 4ms/sample - loss: 0.6162 - accuracy: 0.8004 - val_loss: 0.7876 - val_accuracy: 0.7286
Epoch 6/10
9535/9535 [==============================] - 41s 4ms/sample - loss: 0.5097 - accuracy: 0.8306 - val_loss: 0.5456 - val_accuracy: 0.8339
Epoch 7/10
9535/9535 [==============================] - 41s 4ms/sample

In [24]:
score = model.evaluate(X_test, y_test, verbose = 0 )
print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

Test Loss:  0.4075072174704315
Test Accuracy:  0.86786914


In [18]:
def findClass(image_path):
    image_1 = cv2.imread(image_path)
    image_1 = cv2.resize(image_1, (227, 227))
    np.array(image_1).reshape(-1, 227, 227, 3)
    image_1 = image_1.astype('float32')
    image_1 /= 255 
    prediction = model.predict(np.array([image_1], np.float32))
    print(prediction)
    return CATEGORIES[np.argmax(prediction)]

In [20]:
testData = [
    r'./Datasets/test-data/bright_dune/ESP_016576_2570_RED-0061.jpg', 
    r'./Datasets/test-data/dark_dune/ESP_018334_2600_RED-0501.jpg',
    r'./Datasets/test-data/slope_streak/ESP_011289_1950_RED-0067.jpg', 
    r'./Datasets/hirise-map-proj-v3_2/slope_streak/ESP_011289_1950_RED-0230.jpg',
    r'./Datasets/hirise-map-proj-v3_2/crater/ESP_011425_1775_RED-0030-r90.jpg'
]

for i in testData:
    print(findClass(i))

[[0.6199938  0.15098467 0.00324011 0.00116524 0.22461616]]
bright_dune
[[3.6552537e-05 1.1862278e-01 8.5816234e-01 3.4936951e-04 2.2828998e-02]]
dark_dune
[[1.1381949e-04 2.3087643e-03 4.3160941e-05 1.6792252e-05 9.9751747e-01]]
slope_streak
[[1.0389861e-04 8.0087222e-04 4.1617048e-07 7.6607947e-08 9.9909472e-01]]
slope_streak
[[4.9574976e-04 7.7928525e-01 1.3917187e-01 2.6682983e-03 7.8378960e-02]]
crater


In [21]:
# creates a HDF5 file 'my_model.h5'
model.save('models/cnn3.h5')
# pickle.dump( model, open( "classification_model_v2.pickle", "wb" ))